# Applications in computational materials science

- Task based computing can also be applied into our regular workflows
- Calculation of energy, most analysis tasks can be parallelized in a task based manner
- In this example, LAMMPS is combined with Dask for some simple calculations

Import necessary modules

In [ ]:
import numpy as np
from lammps import lammps
from dask import delayed
from dask.distributed import Client
from ase.io import read, write
from ase.build import bulk
import matplotlib.pyplot as plt
import dask
import os
import warnings
warnings.filterwarnings("ignore")

## EV curves using Cu EAM potential

A function to calculate the lattice constant at 0 pressure using a copper EAM potential. The LAMMPS python library is used for the calculations 

In [ ]:
def find_alat(structure):
    #create lammps object
    lmp = lammps()
    
    #set units, boundary conditions
    lmp.command("echo log")
    lmp.command("units metal")
    lmp.command("atom_style atomic")
    lmp.command("boundary p p p")
    
    #create lattice
    lmp.command("lattice %s %d"%(structure, 4.0))
    lmp.command("region box block 0 2 0 2 0 2")
    lmp.command("create_box 1 box")
    lmp.command("create_atoms 1 box")
    lmp.command("mass * 63.546")
    
    #set the potential
    lmp.command("pair_style eam/alloy")
    lmp.command("pair_coeff * * Cu01.eam.alloy Cu")
    
    #relax the box
    lmp.command("fix 1 all box/relax iso 0. vmax 0.0001 nreset 1")
    lmp.command("minimize 1.0e-8 1.0e-8 100000000 100000000")
    
    #get the lattice constant
    lat= (lmp.get_thermo("vol")/(2*2*2))**(1/3)
    lmp.close()
    return lat

Now another function to calculate the rigid energy at the required lattice constant.

In [ ]:
def calculate_energy(structure, lattice_constant):
    #create lammps object
    lmp = lammps()
    
    #set up units, boundary
    lmp.command("echo log")
    lmp.command("units metal")
    lmp.command("atom_style atomic")
    lmp.command("boundary p p p")
    
    #create lattice
    lmp.command("lattice %s %f"%(structure, lattice_constant))
    lmp.command("region box block 0 2 0 2 0 2")
    lmp.command("create_box 1 box")
    lmp.command("create_atoms 1 box")
    lmp.command("mass * 63.546")
    
    #set up potential
    lmp.command("pair_style eam/alloy")
    lmp.command("pair_coeff * * Cu01.eam.alloy Cu")
    
    #calculate energy
    lmp.command("run 0")
    
    #get cohesive energy
    ecoh = lmp.get_thermo("pe")/lmp.get_natoms()
    return ecoh

Lets test the function

In [ ]:
calculate_energy("fcc", 3.5)

Find lattice constant for fcc structure

In [ ]:
fcclat = find_alat("fcc")
fcclat

Now for bcc

In [ ]:
bcclat = find_alat("bcc")
bcclat

Also simple cubic

In [ ]:
sclat = find_alat("sc")
sclat

Now create arrays over +/- 10% of the lattice constant values for each structure

In [ ]:
bcc_alats = np.linspace(.9*bcclat, 1.1*bcclat, 10)
fcc_alats = np.linspace(.9*fcclat, 1.1*fcclat, 10)
sc_alats = np.linspace(.9*sclat, 1.1*sclat, 10)

In general, we would loop over each lattice constant array and then calculate the energy. But remember, for loops are an excellent choice for task based parallelisation.

Create a local cluster

In [ ]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=1, n_workers=2)
client

Submit the calculations: The `client.map` method will be used. It maps the required function over a number of arguments. 

In [ ]:
future_fcc = client.map(calculate_energy, 
                        ["fcc" for x in range(len(fcc_alats))],
                        fcc_alats)

The `future_fcc` object is not yet a result, but a promise of one.

In [ ]:
future_fcc[0]

We can do the same for bcc and hcp

In [ ]:
future_bcc = client.map(calculate_energy, 
                        ["bcc" for x in range(len(bcc_alats))],
                        bcc_alats)

In [ ]:
future_sc = client.map(calculate_energy, 
                        ["sc" for x in range(len(sc_alats))],
                        sc_alats)

`client.gather` method can be used to gather the results.

In [ ]:
efcc = client.gather(future_fcc)
ebcc = client.gather(future_bcc)
esc = client.gather(future_sc)

Finally plot

In [ ]:
plt.plot((fcc_alats)**3/4, efcc, 'o-', color="#EF6C00", label="fcc")
plt.plot((bcc_alats)**3/2, ebcc, 'o-', color="#AD1457", label="bcc")
plt.plot((sc_alats)**3, esc, 'o-', color="#311B92", label="sc")
plt.legend()
plt.xlabel("vol/atom $\mathrm{\AA^3}$")
plt.ylabel("E (eV)")

What happens if we recalculate one of the arrays

In [ ]:
future_sc = client.map(calculate_energy, 
                        ["sc" for x in range(len(sc_alats))],
                        sc_alats)

Nothing happened in the progress bar! Dask assumes functions are pure - if the function and arguments are same, the result must be same. Hence it does not recalculate. However, you can turn off the pure behaviour.

In [ ]:
future_sc = client.map(calculate_energy, 
                        ["sc" for x in range(len(sc_alats))],
                        sc_alats, pure=False)

In [ ]:
client.restart()

Now the calculations were executed again.

## Parameter phase-diagram for a simple potential

A Stillinger-Weber potential is explored here. The total energy in the framework of SW can be given by,

$$
E = \sum_i\sum_{j>i} \phi_2(r_{ij}) + \sum_i\sum_{j\neq i}\sum_{k> j} \phi_3(r_{ij}, r_{ik}, \theta_{ijk})
$$

Where  $\phi_2$ is,

$$
\phi_2(r_{ij}) = U^R(r_{ij}) - U^A(r_{ij})
$$

$U^R$ is the repulsion and $U^A$ is the attractive term.

$$
U^R(r_{ij}) = A_{ij} B_{ij} \epsilon_{ij} \bigg( \frac{\sigma_{ij}}{r_{ij}} \bigg)^{p_{ij}} \exp \bigg( \frac{\sigma_{ij}}{r_{ij} - a_{ij} \sigma_{ij}} \bigg)
$$
$$
U^A(r_{ij}) = A_{ij} \epsilon_{ij} \bigg( \frac{\sigma_{ij}}{r_{ij}} \bigg)^{q_{ij}} \exp \bigg( \frac{\sigma_{ij}}{r_{ij} - a_{ij} \sigma_{ij}} \bigg)
$$


In addition, SW potential has a three body contribution designed to stabilize the diamond structure:

$$
\phi_3(r_{ij}, r_{ik}, \theta_{ijk}) = 
    \phi_{ijk}
    \exp \bigg( \frac{\gamma_{ij} \sigma_{ij}}{r_{ij} - a_{ij} \sigma_{ij}} \bigg) 
    \exp \bigg( \frac{\gamma_{ik} \sigma_{ik}}{r_{ik} - a_{ik} \sigma_{ik} }\bigg)
$$

where $\phi_{ijk}$ is given by,

$$
\phi_{ijk} = \lambda_{ijk} \epsilon_{ijk} [\cos\theta_{ijk} - \cos\theta_{0ijk}]^2
$$

It penalises the non-tetrahedral angle

### Explore the relative structural stability with potenatial parameters

A small script to write SW potential files for LAMMPS with the given parameters is used.

In [ ]:
from sw import Sw

Now a function to relax the structure and calculate the energy is defined

In [ ]:
def get_energy(l, b, structure, alat):
    #create the SW class and write a potential file
    sw = Sw("Si", lmbda=l, B=b)
    potfile = "sw-%f-%f.sw"%(l, b)
    sw.write(potfile)
    
    #create lammps object, set units and boundary
    lmp = lammps()
    lmp.command("echo log")
    lmp.command("units metal")
    lmp.command("atom_style atomic")
    lmp.command("boundary p p p")
    
    #create lattice
    lmp.command("lattice %s %d"%(structure, alat))
    lmp.command("region box block 0 2 0 2 0 2")
    lmp.command("create_box 1 box")
    lmp.command("create_atoms 1 box")
    lmp.command("mass * 28")
    
    #set up the potential
    lmp.command("pair_style sw")
    lmp.command("pair_coeff * * %s Si"%potfile)
    
    #minimise the structure
    lmp.command("fix 1 all box/relax iso 0. vmax 0.0001 nreset 1")
    lmp.command("minimize 1.0e-8 1.0e-8 100000000 100000000")
    lat= (lmp.get_thermo("vol")/(2*2*2))**(1/3)
    
    #calculate the energy of the relaxed structure
    ecoh = lmp.get_thermo("pe")/lmp.get_natoms()
    lmp.close()
    os.remove(potfile)
    return ecoh

The relative stability will be explored over two parameters:

- $\lambda$ : strength of the three-body contribution
- $B$ : strength of the repulsive interaction

Arrays for $\lambda$ and $B$:

In [ ]:
nlevels = 10
lambda_range = np.linspace(10, 30, nlevels)
B_range = np.linspace(0.6, 1.0, nlevels)

Similar to our previous examples, we will run a for loop - but will not evaluate it. Only a recipe will be created.

In [ ]:
resfcc = []
resdia = []
resbcc = []

for l in lambda_range:
    resbcc.append([delayed (get_energy)(l, b, "bcc", 4.0) for b in B_range])
    resfcc.append([delayed (get_energy)(l, b, "fcc", 4.0) for b in B_range])
    resdia.append([delayed (get_energy)(l, b, "diamond", 5.4) for b in B_range])

We do a final delayed to convert it an array

In [ ]:
enfcc = dask.compute(*resfcc)

Compute for the other structures

In [ ]:
enbcc = dask.compute(*resbcc)
endia = dask.compute(*resdia)

In [ ]:
client.close()

Take a look at the bcc energy

In [ ]:
plt.contourf(B_range, lambda_range, enbcc)
plt.colorbar()
plt.xlabel("$B$", fontsize=12)
plt.ylabel("$\lambda$", fontsize=12)

Convert to energy differences

In [ ]:
edian = np.array(endia)-np.array(endia)
ebccn = np.array(enbcc)-np.array(endia)
efccn = np.array(enfcc)-np.array(endia)

At each value of $B$ and $\lambda$, find which structure has the least energy

In [ ]:
estruct = np.zeros((nlevels, nlevels))
ecomben = np.zeros((nlevels, nlevels))

In [ ]:
for i in range(nlevels):
    for j in range(nlevels):
        earr = [edian[i,j], ebccn[i,j], efccn[i,j]]
        args = np.argsort(earr)
        estruct[i,j] = args[0]
        ecomben[i,j] = earr[args[0]]

Create a numpy meshgrid for plotting

In [ ]:
bb, ll = np.meshgrid(B_range, lambda_range)

Finally plot

In [ ]:
plt.contourf(bb, ll, estruct, levels=2, 
             colors=["#fdc086", "#beaed4", "#7fc97f"])
plt.plot([], [], color="#fdc086", label="diamond")
plt.plot([], [], color="#beaed4", label="bcc")
plt.plot([], [], color="#7fc97f", label="fcc")

plt.xlabel("$B$", fontsize=12)
plt.ylabel("$\lambda$", fontsize=12)
plt.legend()